[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for bodies

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [1]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction_bodies.sh http://josd.github.io/eye/transduction/observation_prediction_bodies.sh
! curl -o observation_prediction_bodies.py http://josd.github.io/eye/transduction/observation_prediction_bodies.py
! curl -o __init__.py http://josd.github.io/eye/transduction/__init__.py
! curl -o sample_bodies.observation http://josd.github.io/eye/transduction/sample_bodies.observation
! chmod +x observation_prediction_bodies.sh
#% rm -fr ~/t2t_data/observation_prediction_bodies
#% rm -fr ~/t2t_train/observation_prediction_bodies/transformer-transformer_small/

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.1)
Requirement not upgraded as not directly required: flask in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.0.2)
Requirement not upgraded as not directly required: scipy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.19.1)
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.11.0)
Requirement not upgraded as not directly required: bz2file in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.98)
Requirement not upgraded as not directly required: gym in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.10.5)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.7.1)
Requirement not upgraded as not directly required: future in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.16.0)
Require

In [2]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_bodies.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionBodies(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for bodies."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 50% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 8,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 2,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(10000):
      # wind turbine size fa

In [3]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_bodies.sh

#!/bin/bash
PROBLEM=observation_prediction_bodies
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=3 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=2000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_bodies.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_bodies.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLE

In [4]:
# Run the observation_prediction_bodies script

! ./observation_prediction_bodies.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_bodies
INFO:tensorflow:Generating data for observation_prediction_bodies.
INFO:tensorflow:Found vocab file: /content/t2t_data/observation_prediction_bodies/vocab.observation_prediction_bodies.16384.subwords
INFO:tensorflow:Skipping generator because outputs files exist
INFO:tensorflow:Skipping shuffle because output files exist
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtyp

INFO:tensorflow:Saving checkpoints for 1501 into /content/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:loss = 0.059572347, step = 1500
INFO:tensorflow:Saving checkpoints for 1600 into /content/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.049057156.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-dev*
INFO:tensorflow:partition: 0 num_data_files: 2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_un

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_754_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-03-09:29:27
INFO:tensorflow:Graph was finalized.
2018-05-03 09:29:27.751690: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-03 09:29:27.751779: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] 

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-03-09:30:16
INFO:tensorflow:Graph was finalized.
2018-05-03 09:30:16.260330: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-03 09:30:16.260423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-03 09:30:16.260458: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-03 09:30:16.260489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-03 09:30:16.260708: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /content/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt-1800
INFO:te

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/3]
INFO:tensorflow:Evaluation [2/3]
INFO:tensorflow:Evaluation [3/3]
INFO:tensorflow:Finished evaluation at 2018-05-03-09:31:09
INFO:tensorflow:Saving dict for global step 1900: global_step = 1900, loss = 0.044366125, metrics-observation_prediction_bodies/accuracy = 0.9947032, metrics-observation_prediction_bodies/accuracy_per_sequence = 0.9714286, metrics-observation_prediction_bodies/accuracy_top5 = 0.9987215, metrics-observation_prediction_bodies/approx_bleu_score = 0.7532912, metrics-observation_prediction_bodies/neg_log_perplexity = -0.091202065, metrics-observation_prediction_bodies/rouge_2_fscore = 0.79873586, metrics-observation_prediction_bodies/rouge_L_fscore = 0.7992327
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:tensorflow:partition: 0 num_data_files: 8
INFO:tensorflow:Cal

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb4a4b848d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': 1234, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.75
}
allow_soft_placement: true
graph_options {
  optimizer_options {
  }
}
, '_save_checkpoints_steps': 1000, '_ke

In [5]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_bodies.observation
print("->-")
! pygmentize -g sample_bodies.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.35	A_PERSON has BMI class O	-4.07	A_PERSON has BMI has BMI class N	-4.97
A_TURBINE producing 9533 kW	-0.69	A_TURBINE producing 3645 kW	-2.06	A_TURBINE producing 14299 kW	-3.42
